# Collecting and Cleaning US Census Data for Various Linguistic Groups#

The purpose of this notebook is to collect two blocks of data from the US Census API Server:

* The geographic distribution for three foreign languages of interest - Spanish, French, and Chinese(1)(2)
* Data on the language groups spoken in Cook County(3)
 
_(1)We originally selected six languages - Spanish, French, Chinese, Hindi, Polish, and Arabic. We focused in on three of them because it is sufficiently illustrative of the Census API's abilities and because there was some unclear directions in the census API documentation and we only figured out how to access Hindi, Polish, and Arabic a few days before the presentation._

_(2)The Census language survey lists "Chinese" as well as Chinese dialects such as Cantonese and Mandarin; the documentation appears to suggest that this is due to the response of the participants. Therefore it appears that Chinese, Cantonese, and Mandarin, for example, are not overlapping groups, they are each separate populations._ 

_(3)The Census collects information on 380 languages. Not all languages are accessible using the basic API query format and for many languages the number of speakers in Cook County is estimated at 'none'. Many languages are not returned by the API request at all. We are at about 150 surveyed languages for cook county._

In [2]:
#Import Functions - all functions in notebook. 

import pandas as pd
import numpy as np
import requests
import json
import csv
from apikey import api_key


## Collecting and cleaning US Census Data for the three target languages nationwide

* This section of the notebook covers the first research question - data about Spanish, Chinese, and French

_Note: The Census does not collect these data for all US counties, only for a number of key metropolitan areas. There are other tiers of analysis - state and nationwide estimates - but we chose to focus on metro areas only._ 

In [2]:

#These are the language codes used by the Census in their API requests. They're from a PDF downloaded from the Census website so there's no way I'm aware of to do this automatically. 

spanish = 625
chinese = 708
french = 620

#This is the census API URL for a search covering all counties in the survey
#It is returning the population estimate, the name of the language, and the name of the geographic area. 

url = "https://api.census.gov/data/2013/language?get=EST,LANLABEL,NAME&for=county:*&LAN="

#These are the API queries for the three languages that return results from the Census

query_spanish = f"{url}{spanish}&key={api_key}"
query_french = f"{url}{french}&key={api_key}"
query_chinese = f"{url}{chinese}&key={api_key}"

In [3]:
#We now send API requests to the census for county data for three languages - English, French, and Chinese

response_f = requests.get(query_french).json()
response_s = requests.get(query_spanish).json()
response_c = requests.get(query_chinese).json()

#Formatting those responses into dataframes 

df_french = pd.DataFrame(response_f)
df_spanish = pd.DataFrame(response_s)
df_chinese = pd.DataFrame(response_c)

#get rid of the default headings (the columns are automatically entitled 1,2,3 etc)
df_french.columns = df_french.iloc[0]
df_french = df_french.drop(df_french.index[0])

df_spanish.columns = df_spanish.iloc[0]
df_spanish = df_spanish.drop(df_spanish.index[0])

df_chinese.columns = df_chinese.iloc[0]
df_chinese = df_chinese.drop(df_chinese.index[0])

#Quick test that this has all worked so far. 
print(len(df_french))
df_french.head()

155


,EST,LANLABEL,NAME,LAN,state,county
1,1035,French,"Anchorage Municipality, AK",620,02,020
2,10680,French,"Maricopa County, AZ",620,04,013
3,85,French,"Navajo County, AZ",620,04,017
4,3230,French,"Pima County, AZ",620,04,019
5,6915,French,"Alameda County, CA",620,06,001


In [23]:
#We need the total populations of each county as of 2013. This comes from a different API at the Census. 
#apparently I can't download all the counties in all the states at once (so I could just merge on FIPS code)
#so I ended up querying the API just for a list of the counties we're already looking at, one by one, using a "for" loop.
#this takes forever and I'm sorry. 

countypoplist = []

#Note that this uses the french dataframe as its source - that shouldn't make a difference (the county lists are the same for all languages surveyed)

for row in df_french.index:
    st = df_french.loc[row, 'state']
    print(st)
    cty = df_french.loc[row, 'county']
    print(cty)
    query_pop = f"https://api.census.gov/data/2013/pep/cty?get=STNAME,POP,CTYNAME&for=county:{cty}&in=state:{st}&DATE_=6&key={api_key}"
    response_pop = requests.get(query_pop).json()
    countypoplist.append(response_pop)

print(countypoplist)

02
020
04
013
04
017
04
019
06
001
06
013
06
019
06
029
06
037
06
053
06
059
06
061
06
065
06
067
06
071
06
073
06
075
06
077
06
081
06
083
06
085
06
095
06
097
06
099
06
111
06
113
08
005
08
031
08
041
09
001
09
003
09
009
10
003
11
001
12
011
12
021
12
031
12
057
12
071
12
086
12
095
12
099
12
103
13
067
13
089
13
121
13
135
15
001
15
003
15
009
17
031
17
043
17
089
17
097
17
197
18
097
19
153
20
091
21
111
22
051
24
003
24
005
24
027
24
031
24
033
24
510
25
005
25
009
25
013
25
017
25
021
25
023
25
025
25
027
26
099
26
125
26
161
26
163
27
037
27
053
27
123
29
189
32
003
32
031
33
011
34
001
34
003
34
005
34
007
34
013
34
017
34
021
34
023
34
025
34
027
34
029
34
031
34
035
34
039
35
001
35
045
36
001
36
005
36
029
36
047
36
055
36
059
36
061
36
067
36
071
36
081
36
085
36
087
36
103
36
119
37
081
37
119
37
183
39
035
39
049
39
061
40
109
41
051
41
067
42
003
42
017
42
029
42
045
42
071
42
091
42
101
44
007
47
037
47
157
48
029
48
085
48
113
48
121
48
157
48
201
48
439
48
453
49
035

### FIPS Codes

* Every county in the United States has a unique five-digit identifier code that lets us look it up. It's called a FIPS Code. 
* We can use this unique FIPS code to merge several different datasets for the same geographic area. 
* Several of the following cells are concerned with generating FIPS codes for the different counties in the survey.
    * The Census sends us the correct number but with the FIPS prefix for the state separate from the FIPS code for the county so they must be combined. 

In [24]:
#this formats all the county populations into a dataframe to be merged below for county populations.


ctylist2 = [item[1] for item in countypoplist]

ctylist2_df = pd.DataFrame(ctylist2)

ctylist2_df.columns = ['STNAME', 'population', 'CTYNAME', 'DATE_', 'state', 'county']

ctylist2_df["fips"] = ctylist2_df["state"].map(str) + ctylist2_df["county"]

ctypop_df = ctylist2_df.drop(['STNAME', 'CTYNAME', 'DATE_', 'state', 'county'], axis=1)

ctypop_df.head()

,population,fips
0,300950,02020
1,4009412,04013
2,107322,04017
3,996554,04019
4,1578891,06001


In [25]:
#Now we're constructing FIPS codes in each of the language dataframes (this will allow us to merge on FIPS code in a second)

df_french["FIPS Code"] = df_french["state"].map(str) + df_french["county"]
df_spanish["FIPS Code"] = df_spanish["state"].map(str) + df_spanish["county"]
df_chinese["FIPS Code"] = df_chinese["state"].map(str) + df_chinese["county"]
#creates a FIPS Code column into each of the language dataframes

#https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-dataframe-in-pandas-python


df_french.head()

,EST,LANLABEL,NAME,LAN,state,county,FIPS Code
1,1035,French,"Anchorage Municipality, AK",620,02,020,02020
2,10680,French,"Maricopa County, AZ",620,04,013,04013
3,85,French,"Navajo County, AZ",620,04,017,04017
4,3230,French,"Pima County, AZ",620,04,019,04019
5,6915,French,"Alameda County, CA",620,06,001,06001


In [45]:
#dropping unecessary columns from the dataframes in preparation for merging
#Keeping state and county codes for sorting list when shading counties
df_french2 = df_french
df_spanish2 = df_spanish
df_chinese2 = df_chinese

#del df_french['county']
df_french2 = df_french.rename(columns={"county":"county code","state":"state code"})
df_french2 = df_french2.rename(columns={'EST': 'no_french', 'LANLABEL': 'Language', 'NAME': 'county', 'FIPS Code': 'fips'})
#print(df_french2)
#del df_french['state']
del df_french2['LAN']
del df_french2['Language']

#del df_spanish['county']
#df_spanish2 = df_spanish.rename(columns={"county":"county code","state":"state code"})
df_spanish2 = df_spanish2.rename(columns={'EST': 'no_spanish', 'LANLABEL': 'Language', 'NAME': 'county', 'FIPS Code': 'fips'})
del df_spanish2['state']
del df_spanish2['LAN']
del df_spanish2['county']
del df_spanish2['Language']

#del df_chinese2['county']
#df_chinese2 = df_chinese.rename(columns={"county":"county code","state":"state code"})
df_chinese2 = df_chinese2.rename(columns={'EST': 'no_chinese', 'LANLABEL': 'Language', 'NAME': 'county', 'FIPS Code': 'fips'})
del df_chinese2['state']
del df_chinese2['LAN']
del df_chinese2['county']
del df_chinese2['Language']

#merge french and spanish into a master dataframe

df_all = df_french2.merge(df_spanish2, how = 'inner', on = 'fips')



In [46]:
#add chinese to the master dataframe

df_all = df_all.merge(df_chinese2, how='inner', on = 'fips')

In [47]:
#merging total county populations into the master dataframe

df_all = df_all.merge(ctypop_df, how='inner', on = 'fips')

df_all.head()

,no_french,county,state code,county code,fips,no_spanish,no_chinese,population
0,1035,"Anchorage Municipality, AK",02,020,02020,12635,740,300950
1,10680,"Maricopa County, AZ",04,013,04013,733630,13750,4009412
2,85,"Navajo County, AZ",04,017,04017,5885,None,107322
3,3230,"Pima County, AZ",04,019,04019,217115,4625,996554
4,6915,"Alameda County, CA",06,001,06001,241150,69165,1578891


In [48]:
#there are values for chinese that say 'none' and i"m trying to make them go away. 
#https://stackoverflow.com/questions/23743460/replace-none-with-nan-in-pandas-dataframe
df_all['no_chinese'].replace(to_replace=[None], value=[0], inplace=True)

#turns out all the numbers are still set as strings so we're setting them to integers...
df_all['no_spanish'] = df_all['no_spanish'].astype(int)
df_all['no_french'] = df_all['no_french'].astype(int)
df_all['no_chinese'] = df_all['no_chinese'].astype(int)
df_all['population'] = df_all['population'].astype(int)

#and now we're calculating the percentages of the population that speak each language...
df_all['pct_spanish'] = (df_all['no_spanish']/df_all['population'])*100
df_all['pct_french'] = (df_all['no_french']/df_all['population'])*100
df_all['pct_chinese'] = (df_all['no_chinese']/df_all['population'])*100

df_all.head()

,no_french,county,state code,county code,fips,no_spanish,no_chinese,population,pct_spanish,pct_french,pct_chinese
0,1035,"Anchorage Municipality, AK",02,020,02020,12635,740,300950,4.198372,0.343911,0.245888
1,10680,"Maricopa County, AZ",04,013,04013,733630,13750,4009412,18.297696,0.266373,0.342943
2,85,"Navajo County, AZ",04,017,04017,5885,0,107322,5.483498,0.079201,0.000000
3,3230,"Pima County, AZ",04,019,04019,217115,4625,996554,21.786577,0.324117,0.464099
4,6915,"Alameda County, CA",06,001,06001,241150,69165,1578891,15.273379,0.437966,4.380606


In [55]:
#reorder the columns in a way that makes more sense. 

df_all = df_all[['fips', 'county', 'state code', 'county code', 'population', 'no_french', 'pct_french', 'no_spanish', 'pct_spanish', 'no_chinese', 'pct_chinese']]
df_all.sort_values(by=['state code', 'county'])

,fips,county,state code,county code,population,no_french,pct_french,no_spanish,pct_spanish,no_chinese,pct_chinese
0,02020,"Anchorage Municipality, AK",02,020,300950,1035,0.343911,12635,4.198372,740,0.245888
1,04013,"Maricopa County, AZ",04,013,4009412,10680,0.266373,733630,18.297696,13750,0.342943
2,04017,"Navajo County, AZ",04,017,107322,85,0.079201,5885,5.483498,0,0.000000
3,04019,"Pima County, AZ",04,019,996554,3230,0.324117,217115,21.786577,4625,0.464099
4,06001,"Alameda County, CA",06,001,1578891,6915,0.437966,241150,15.273379,69165,4.380606
...,...,...,...,...,...,...,...,...,...,...,...
150,53033,"King County, WA",53,033,2044449,11955,0.584754,121930,5.963954,38030,1.860159
151,53053,"Pierce County, WA",53,053,819743,1825,0.222631,44005,5.368146,1665,0.203112
152,53061,"Snohomish County, WA",53,061,745913,2205,0.295611,41660,5.585102,5065,0.679034
153,55025,"Dane County, WI",55,025,509939,2050,0.402009,23715,4.650556,4430,0.868731


In [56]:
#export it all to a CSV called "langs by county" for use in the analysis portion. 

df_all.to_csv('langs_by_county.csv', index=False)

## Collecting and cleaning US Census Data for all languages spoken in Cook County

* This section of the notebook covers the second research question - how many language speakers are in cook county? 

_Note: Remember that only about 155 languages are covered by the Census and the rationale behind their selection doesn't make a ton of sense to us as outside observers._



In [3]:
#the language codes used by the census range from 600 to 999. I am setting up a "for" loop that will ping the API server for each of these 500 codes in turn.
#We'll then put the codes that return values into a DataFrame for subsequent use. 
#Be warned that this takes about five minutes to complete. 

numbers = np.arange(600,999)

langlist = []

for value in numbers:
    try: 
        line = requests.get(f"https://api.census.gov/data/2013/language?get=EST,LANLABEL,LAN39&for=county:031&in=state:17&LAN={value}&key={api_key}").json()
        print(line)
        langlist.append(line)
    except:
        print(f"No response from server for {value}")

#These are some quick formats to the dataframe. 
        
langlist = [item[1] for item in langlist]

df_cook = pd.DataFrame(langlist, columns = ["Speakers", "Language", "Lang_39", "Lang_Code", "State", "County"])

df_cook.head()

No response from server for 600
[['EST', 'LANLABEL', 'LAN39', 'LAN', 'state', 'county'], ['555', 'Jamaican Creole', '00', '601', '17', '031']]
[['EST', 'LANLABEL', 'LAN39', 'LAN', 'state', 'county'], [None, 'Krio', '00', '602', '17', '031']]
No response from server for 603
[['EST', 'LANLABEL', 'LAN39', 'LAN', 'state', 'county'], ['50', 'Pidgin', '00', '604', '17', '031']]
No response from server for 605
No response from server for 606
[['EST', 'LANLABEL', 'LAN39', 'LAN', 'state', 'county'], ['17305', 'German', '00', '607', '17', '031']]
No response from server for 608
[['EST', 'LANLABEL', 'LAN39', 'LAN', 'state', 'county'], ['1154', 'Yiddish', '07', '609', '17', '031']]
[['EST', 'LANLABEL', 'LAN39', 'LAN', 'state', 'county'], ['1550', 'Dutch', '00', '610', '17', '031']]
[['EST', 'LANLABEL', 'LAN39', 'LAN', 'state', 'county'], ['115', 'Afrikaans', '00', '611', '17', '031']]
No response from server for 612
No response from server for 613
[['EST', 'LANLABEL', 'LAN39', 'LAN', 'state', 'cou

,Speakers,Language,Lang_39,Lang_Code,State,County
0,555,Jamaican Creole,00,601,17,031
1,None,Krio,00,602,17,031
2,50,Pidgin,00,604,17,031
3,17305,German,00,607,17,031
4,1154,Yiddish,07,609,17,031


In [4]:
#This exports all the languages in Cook County to a CSV for use in the analysis program. 

df_cook.to_csv('cook_languages.csv', index=True)